<a href="https://colab.research.google.com/github/orlandobeco/TFG_ML_CMAPSS/blob/main/TFG_Orlando_CMAPSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TFG Completo

In [ ]:
# LIBERAR DOWNLOAD NO 
import time
from google.colab import files
files.download('/content/sample_data/README.md')
files.download('/content/sample_data/anscombe.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Baixando Base

In [ ]:
import gdown
#https://drive.google.com/file/d/1M5Bo2sdK99OqI3Z7BhSVS_gB3auMF1X6/view?usp=sharing
url = 'https://drive.google.com/uc?id=1M5Bo2sdK99OqI3Z7BhSVS_gB3auMF1X6'
output = 'N-CMAPSS_DS02-006.h5'
gdown.download(url, output, quiet=False)
filename = 'N-CMAPSS_DS02-006.h5'

Downloading...
From: https://drive.google.com/uc?id=1M5Bo2sdK99OqI3Z7BhSVS_gB3auMF1X6
To: /content/N-CMAPSS_DS02-006.h5
100%|██████████| 2.45G/2.45G [00:28<00:00, 85.8MB/s]


## Leitura do Dataset

In [ ]:
import h5py
import numpy as np
import pandas as pd

# Load data
with h5py.File(filename, 'r') as hdf:
  W = np.concatenate((
      np.array(hdf.get('W_dev')), 
      np.array(hdf.get('W_test')),
      ), axis=0)
  
  X_s = np.concatenate((
      np.array(hdf.get('X_s_dev')), 
      np.array(hdf.get('X_s_test')),
      ), axis=0)
  
  X_v = np.concatenate((
      np.array(hdf.get('X_v_dev')), 
      np.array(hdf.get('X_v_test')),
      ), axis=0)
  
  Y = np.concatenate((
      np.array(hdf.get('Y_dev')), 
      np.array(hdf.get('Y_test')),
      ), axis=0)

  #Nome das colunas
  W_var = np.array(hdf.get('W_var'))
  X_s_var = np.array(hdf.get('X_s_var'))  
  X_v_var = np.array(hdf.get('X_v_var')) 
  #corrigindo nomes
  W_var = list(np.array(W_var, dtype='U20'))
  X_s_var = list(np.array(X_s_var, dtype='U20'))  
  X_v_var = list(np.array(X_v_var, dtype='U20'))

#agrupando dados
data = np.concatenate((W, X_s, X_v, Y), axis=1)
columns = np.concatenate((W_var, X_s_var, X_v_var, ['RUL']), axis=0)

#eliminando da memória vetores não mais usados
del W, X_s, X_v, Y

#criando dataset final
ds = pd.DataFrame(data=data, columns=columns)

#eliminando da memória copia
del data, columns

#formato do dataset
ds.shape

(6517190, 33)

In [ ]:
ds.head()

,alt,Mach,TRA,T2,T24,T30,T48,T50,P15,P2,P21,P24,Ps30,P40,P50,Nf,Nc,Wf,T40,P30,P45,W21,W22,W25,W31,W32,W48,W50,SmFan,SmLPC,SmHPC,phi,RUL
0,10005.0,0.448497,76.903748,502.420918,600.148034,1438.498187,1818.027714,1228.129848,15.806267,11.577097,16.046971,20.126624,331.293679,336.631827,12.629361,2160.926416,8591.373490,3.855337,2797.727106,352.494060,62.768751,2420.557738,228.486927,228.487065,26.498785,15.899271,215.844851,228.411666,16.648833,9.898130,25.376144,41.893990,74.0
1,10013.0,0.447741,76.903748,502.326114,600.055894,1438.350208,1817.682618,1227.879113,15.795477,11.568235,16.036017,20.113218,331.109867,336.446748,12.623033,2160.909333,8590.972866,3.852319,2797.209328,352.300260,62.734717,2418.935113,228.382424,228.383505,26.486552,15.891931,215.745634,228.307014,16.639222,9.904927,25.380549,41.884434,74.0
2,10017.0,0.448938,77.079529,502.416067,600.210756,1439.109101,1820.020627,1229.422522,15.807747,11.574866,16.048474,20.130956,331.753181,337.082502,12.637951,2161.819062,8593.031745,3.866201,2800.736255,352.965971,62.851755,2421.245669,228.651138,228.661083,26.519340,15.911604,216.019054,228.592279,16.649823,9.923503,25.318848,41.953848,74.0
3,10024.0,0.449883,77.079529,502.469893,600.369717,1439.240230,1819.188327,1228.538726,15.816360,11.578198,16.057218,20.146716,331.819136,337.162828,12.631509,2162.768666,8593.781545,3.863328,2799.352383,353.050081,62.873081,2422.941615,228.786949,228.768625,26.532044,15.919226,216.121238,228.702994,16.653812,9.905518,25.361981,41.914342,74.0
4,10031.0,0.449379,77.079529,502.401271,600.298227,1439.064004,1818.963540,1228.389046,15.807513,11.571593,16.048236,20.135888,331.626003,336.966936,12.624872,2162.619544,8593.220200,3.860818,2799.034785,352.844959,62.836600,2421.608933,228.651685,228.653631,26.518460,15.911076,216.008509,228.584788,16.649031,9.897465,25.363994,41.911503,74.0


## Treinamento

## Metodologias de Aprendizado Regressivo

In [ ]:
regressores=[]
reg_nomes =[]

### Lineares

In [ ]:
from sklearn import linear_model

regressores.append(linear_model.LinearRegression())
reg_nomes.append('LinearRegression')

regressores.append(linear_model.Ridge(alpha=1.0))
reg_nomes.append('Ridge')

regressores.append(linear_model.Lasso(alpha=0.1))
reg_nomes.append('Lasso')

regressores.append(linear_model.ElasticNet(random_state=42))
reg_nomes.append('ElasticNet')

#regressores.append(linear_model.LogisticRegression(random_state=42))
#reg_nomes.append('LogisticRegression')

### Não Lineares

In [ ]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

#regressores.append(KernelRidge(alpha=1.0,kernel='rbf'))
#reg_nomes.append('KernelRidge')

#regressores.append(SVR(kernel='rbf'))
#reg_nomes.append('SVM-R')

#regressores.append(DecisionTreeRegressor(random_state=42,max_depth=10))
#reg_nomes.append('DecisionTreeRegressor')

regressores.append(MLPRegressor(
    random_state=42,
    hidden_layer_sizes=(100,),
    activation='relu',
    solver='adam',
    ))
reg_nomes.append('MLPRegressor')

## Métrica

In [ ]:
from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

## Pipeline

### Completa

In [ ]:
X = ds[ds.columns[:-1]].values
y = ds[ds.columns[-1]].values
print(X.shape,y.shape)

(6517190, 32) (6517190,)


### Base reduzida para Debug

In [ ]:
#X = ds[ds.columns[:-1]].sample(n=1000000,random_state=42).values
#y = ds[ds.columns[-1]].sample(n=1000000,random_state=42).values
#print(X.shape,y.shape)

#X = ds[ds.columns[:-1]].sample(n=700000,random_state=42).values
#y = ds[ds.columns[-1]].sample(n=700000,random_state=42).values
#print(X.shape,y.shape)

### Treinamento

In [ ]:
del ds

In [ ]:
import time
from sklearn.model_selection import KFold

#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
num_partes = 5
kf = KFold(n_splits=num_partes, random_state=42, shuffle=True) # 10-Fold

resultados =[]
res_nomes = ['Técnica','Tempo(s)','ME','MAE','MSE','R2']

for nome, rgr in zip(reg_nomes, regressores):
  print('==',nome,'==')
  for train_index, test_index in kf.split(X):
    etapa = []
    etapa.append(nome)

    # dataset de treinamento(X_train,y_train)
    # dataset de testes(X_test,y_test)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    #treinamento
    t = time.process_time()  
    rgr.fit(X_train,y_train)
    etapa.append(time.process_time()-t)
    
    #validacao
    y_prev = rgr.predict(X_test)

    #métricas
    etapa.append(max_error(y_test,y_prev))
    etapa.append(mean_absolute_error(y_test,y_prev))
    etapa.append(mean_squared_error(y_test,y_prev))
    etapa.append(r2_score(y_test,y_prev))

    print(etapa)

    resultados.append(etapa)
    #armazenando resultados
    res1 = pd.DataFrame(data = resultados, columns =res_nomes )
    # salva
    res1.to_excel('sem_pca.xlsx',index=True)

#armazenando resultados
res1 = pd.DataFrame(data = resultados, columns =res_nomes )
# salva
res1.to_excel('sem_pca.xlsx',index=True)

from google.colab import files
files.download('sem_pca.xlsx')

== LinearRegression ==
['LinearRegression', 18.11080481, 292.43358102216735, 9.397005614938584, 129.20680761262514, 0.7301230518092502]
['LinearRegression', 16.571106953000005, 319.4746339437552, 9.400625069114318, 129.52027426713474, 0.7295840589476049]
['LinearRegression', 15.510009526999994, 214.1959545519203, 9.396524056153007, 129.172775960075, 0.7301896734469928]
['LinearRegression', 16.067478955, 280.936596293468, 9.400310478763423, 129.30934006685575, 0.7300406663512435]
['LinearRegression', 15.907182399999996, 281.93736284866463, 9.409527242106789, 129.6303718555391, 0.729969850722137]
== Ridge ==
['Ridge', 2.764552883999997, 280.0897209293198, 9.401438190192264, 129.2436244973746, 0.7300461516158188]
['Ridge', 2.73146483699999, 309.67314837691447, 9.405358458240407, 129.55490413534812, 0.7295117577696277]
['Ridge', 2.678065601, 196.98502408147124, 9.401147603053852, 129.2123177398974, 0.7301070803429346]
['Ridge', 2.7177785929999914, 268.70336558624876, 9.405114562637111, 129

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.915e+08, tolerance: 2.498e+05
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


['Lasso', 1009.1832806799999, 618.7355559715979, 9.978787761234788, 146.99084665477025, 0.6929771593297405]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.910e+08, tolerance: 2.498e+05
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


['Lasso', 1011.4440568949999, 617.7422372347833, 9.985398113453105, 147.827040697944, 0.6913626955353918]


...

['ElasticNet', 868.8895034620009, 549.424810646076, 10.100680462746118, 151.06401467427327, 0.6846233944223361]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.871e+08, tolerance: 2.497e+05
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


['ElasticNet', 885.9251757679995, 520.8580371216187, 10.110900452380864, 151.26597782624765, 0.6849012003251599]
== DecisionTreeRegressor ==
['DecisionTreeRegressor', 282.0183152890004, 76.21493268053855, 8.942909163956081, 152.38735190206478, 0.681705366504384]
['DecisionTreeRegressor', 282.55250330499985, 76.50420168067227, 8.897709019755206, 151.23365619871205, 0.6842502713094833]
['DecisionTreeRegressor', 280.25798388600015, 72.64871928907475, 8.86873193858772, 150.3895673359219, 0.6858729870014402]
['DecisionTreeRegressor', 279.83034738400056, 69.28168677577351, 8.92206087718993, 151.6607636812204, 0.6833775604849908]
['DecisionTreeRegressor', 279.6009638730011, 77.13853307597824, 8.883804366797149, 150.9868770225325, 0.6854825890121781]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Treinamento com pré-processamento

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#limita escala dos atributos//colunas
scaler = StandardScaler()
X = scaler.fit_transform(X)

# decompoem informação de 33 atr. para 2 atr/col
pca = PCA(n_components=2)
X = pca.fit_transform(X)

import time
from sklearn.model_selection import KFold

#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
num_partes = 5
kf = KFold(n_splits=num_partes, random_state=42, shuffle=True) # 10-Fold

resultados =[]
res_nomes = ['Técnica','Tempo(s)','ME','MAE','MSE','R2']

for nome, rgr in zip(reg_nomes, regressores):
  print('==',nome,'==')
  for train_index, test_index in kf.split(X):
    etapa = []
    etapa.append(nome)

   # dataset de treinamento(X_train,y_train)
   # dataset de testes(X_test,y_test)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    #treinamento
    t = time.process_time()  
    rgr.fit(X_train,y_train)
    etapa.append(time.process_time()-t)
    
    #validacao
    y_prev = rgr.predict(X_test)

    #métricas
    etapa.append(max_error(y_test,y_prev))
    etapa.append(mean_absolute_error(y_test,y_prev))
    etapa.append(mean_squared_error(y_test,y_prev))
    etapa.append(r2_score(y_test,y_prev))

    print(etapa)

    resultados.append(etapa)

#armazenando resultados
res1 = pd.DataFrame(data = resultados, columns =res_nomes )
# salva
res1.to_excel('com_pca.xlsx',index=True)

from google.colab import files
files.download('com_pca.xlsx')

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


['MLPRegressor', 3160.652487843, 56.42552628786217, 18.444117786369898, 469.0238568501235, 0.020340108588844275]


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


['MLPRegressor', 418.4182368470001, 55.506204959196936, 18.50684664962311, 471.3810358394907, 0.015837890075149463]


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


...

['MLPRegressor', 17.4749463940002, 53.47798645597053, 18.66747507003562, 477.2688652113818, 0.003604961891012337]


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


['MLPRegressor', 2.506541630999891, 65.97771226987989, 19.48808884910382, 534.2010234130208, -0.11278229038298737]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Treinamento Incremental

https://scikit-learn.org/0.15/modules/scaling_strategies.html

In [ ]:
import time
from sklearn.model_selection import KFold
from sklearn.linear_model import SGDRegressor, PassiveAggressiveRegressor

from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

resultados = []

incrementais = (SGDRegressor(), PassiveAggressiveRegressor())

X = ds[ds.columns[:-1]].values
y = ds[ds.columns[-1]].values

kf = KFold(n_splits=3, random_state=42, shuffle=True)

for inc in incrementais:
  print('==',inc.__class__,'==')
  for train_index, test_index in kf.split(X):
    etapa = []
    etapa.append(inc.__class__)

    train_size = len(train_index)
    train_step = int(train_size/100)
    
    t = time.process_time()

    for step in range(1, train_size, train_step):
      train_baund = train_index[step:step+train_step]
      X_train = X[train_baund]
      y_train = y[train_baund]

      inc.partial_fit(X_train,y_train)


    etapa.append(time.process_time()-t)
    
    X_test = X[test_index]
    y_test = y[test_index]

    #validacao
    y_prev = inc.predict(X_test)

    #métricas
    etapa.append(max_error(y_test,y_prev))
    etapa.append(mean_absolute_error(y_test,y_prev))
    etapa.append(mean_squared_error(y_test,y_prev))
    etapa.append(r2_score(y_test,y_prev))

    print(etapa)

    resultados.append(etapa)
    #armazenando resultados
    res1 = pd.DataFrame(data = resultados)
    # salva
    res1.to_excel('incremental.xlsx',index=True)

#armazenando resultados
res1 = pd.DataFrame(data = resultados )

== <class 'sklearn.linear_model._stochastic_gradient.SGDRegressor'> ==
[<class 'sklearn.linear_model._stochastic_gradient.SGDRegressor'>, 1.6484487250000086, 1.780015846100149e+17, 1.0071805517214408e+17, 1.1437124231022273e+34, -2.3883172195373737e+31]
[<class 'sklearn.linear_model._stochastic_gradient.SGDRegressor'>, 1.5247446579999746, 8.951021291035618e+16, 5.459117901646967e+16, 3.243375892596383e+33, -6.773115668905087e+30]
[<class 'sklearn.linear_model._stochastic_gradient.SGDRegressor'>, 1.5487041450000163, 1.0101040104997694e+17, 6.072212469075741e+16, 4.0386934828513994e+33, -8.421224750065319e+30]
== <class 'sklearn.linear_model._passive_aggressive.PassiveAggressiveRegressor'> ==
[<class 'sklearn.linear_model._passive_aggressive.PassiveAggressiveRegressor'>, 1.4151834410000106, 112.14404123189856, 25.063453767854575, 991.0661202061708, -1.069558949240275]
[<class 'sklearn.linear_model._passive_aggressive.PassiveAggressiveRegressor'>, 1.4927175680000175, 171.18285316890788, 2